### Comparação de Histograma

In [ ]:
import numpy as np
import glob
import cv2
import os
import pandas as pd

# Inicialize o dicionário de índice para armazenar o nome da imagem, os histogramas correspondentes e o dicionário de imagens para armazenar as próprias imagens

index = {}  # Armazenar o nome da imagem e os histogramas
images = {} # Armazenar as próprias imagens

# Pegar as imagens na pasta
for imagePath in glob.glob(os.getcwd() + "\\images\\hist_compare\\*.jpg"):

    # Extrair o nome do arquivo da imagem (considerado único) e carregar a imagem, atualizando o dicionário de imagens
    filename = imagePath[imagePath.rfind("/") + 1:]
    image = cv2.imread(imagePath)
    images[filename] = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Extrair um histograma de cores RGB da imagem, usando 8 caixas por canal, normalizar e atualizar o índice
    hist = cv2.calcHist([image], [1,2], None, [8, 8],[0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    index[filename] = hist

In [ ]:
# Verificar o calculo do histograma
index

### Métodos para comparar histogramas

Função: **cv2.compareHist(H1, H2, método)**
1. **H1** = primeiro histograma a ser comparado
2. **H2** = segundo histograma a ser comparado
3. **método** = uma flag que indica qual método de comparação deve ser executado. A flag pode ser de qualquer um dos seguintes métodos:
    - **cv2.HISTCMP_CORREL**: calcula a correlação entre os dois histogramas
    - **cv2.HISTCMP_CHISQR**: calcula a distância de chi-quadrado entre os histogramas
    - **cv2.HISTCMP_INTERSECT**: calcula a interseção entre os histogramas
    - **cv2.HISTCMP_BHATTACHARYYA**: distância de Bhattacharyya, usada para medir "sobreposição" entre os dois histogramas


In [ ]:
# Metodos para cálculo do histograma
OPENCV_METHODS = (("Correlation", cv2.HISTCMP_CORREL), ("Chi-Squared", cv2.HISTCMP_CHISQR), ("Intersection", cv2.HISTCMP_INTERSECT), ("Bhattacharyya", cv2.HISTCMP_BHATTACHARYYA))

imagem_analisada = '\\images\\hist_compare\\apple1.jpg'

lista_resultados = []
lista_methodName = []

for (methodName, method) in OPENCV_METHODS:

    results = {}
    reverse = False

    # Se estivermos usando a Correlation ou Intersection, classificar os resultados na ordem inversa
    if methodName in ("Correlation", "Intersection"):
        reverse = True

    for (k, hist) in index.items():
        # Calcular a distancia entre os dois histogramas usando os metodos
        # Atualizar o dicionario de resultados
        d = cv2.compareHist(index[os.getcwd() + imagem_analisada], hist, method)
        results[k] = d

    # Ordenar os resultados
    lista_methodName.append(methodName)
    results = sorted([(v, k) for (k, v) in results.items()], reverse = reverse)
    lista_resultados.append(results)

In [ ]:
lista_resultados

In [ ]:
# Criar o dataframe

for i in range(len(lista_methodName)):
    lista_resultados[i] =  pd.DataFrame(lista_resultados[i])
    lista_resultados[i]['Metodo'] = lista_methodName[i]


df = pd.concat(lista_resultados)
df.to_csv('Resultado_compareHist.csv', sep=';', encoding='latin1')

In [ ]:
# Analise do DataFrame
# Correlacao
correlation = df.loc[df['Metodo'] == 'Correlation']
correlation.sort_values(0)

for i in range(len(correlation)):
    print(correlation[1][i])

In [ ]:
#Chi-Quadrado
ChiSquared = df.loc[df['Metodo'] == 'Chi-Squared']
ChiSquared.sort_values(0)

for i in range(len(ChiSquared)):
    print(ChiSquared[1][i])

In [ ]:
#Intersecao
Intersection = df.loc[df['Metodo'] == 'Intersection']
Intersection.sort_values(0)

for i in range(len(Intersection)):
    print(Intersection[1][i])

In [ ]:
#Bhattacharyya
Bhattacharyya = df.loc[df['Metodo'] == 'Bhattacharyya']
Bhattacharyya.sort_values(0)

for i in range(len(Bhattacharyya)):
    print(Bhattacharyya[1][i])